# 4.3 基于llama的基因数据词典扩充

前面介绍了huggingface自带的分词器构建代码，这里介绍下更为通用的sentencepiece，部分huggingface其实就是来自于这个框架。

SentencePiece 是一个语言无关的分词框架，由 Google 开发并开源。它不同于传统的基于词汇表（如词典）的分词方法，而是采用一种无监督的学习方式来训练模型，从而将文本分割成“子词”单元（subword units）。这种方法使得 SentencePiece 在处理未知词、罕见词以及多语言文本时表现出色。

### 主要特点

1. **语言无关**：
   - SentencePiece 不依赖于任何特定语言的规则或词典，因此它可以应用于任何语言，甚至是混合语言的文本。

2. **子词分词**：
   - 它生成的是子词级别的 token，而不是完整的单词。这种方式可以有效地处理 OOV (out-of-vocabulary) 问题，并且有助于减少词汇表的大小。

3. **无监督学习**：
   - SentencePiece 使用无监督的方法从原始文本中学习分词规则，这意味着你只需要提供未标注的文本数据即可训练分词模型。

4. **灵活的分词粒度**：
   - 可以通过调整参数控制分词的粒度，即生成的子词单元的平均长度。这允许根据具体应用需求优化性能。

5. **支持 BPE 和 Unigram LM**：
   - SentencePiece 实现了两种流行的分词算法：Byte Pair Encoding (BPE) 和 Unigram Language Model (Unigram LM)。这两种方法各有优劣，可以根据任务选择合适的一种。

6. **易于集成**：
   - 提供了多种编程语言的绑定，包括 Python、C++、Go 等，方便在不同环境中使用。

### 工作流程

1. **准备语料库**：
   - 收集用于训练分词模型的未标注文本数据。

2. **训练模型**：
   - 使用 `sentencepiece_trainer` 工具对收集到的文本进行训练，生成分词模型文件。
   ```bash
   spm_train --input=your_corpus.txt --model_prefix=myprefix --vocab_size=8000
   ```

3. **编码和解码**：
   - 训练完成后，可以使用生成的模型对新文本进行编码（分词）和解码（还原）。
   ```python
   import sentencepiece as spm

   # 加载训练好的模型
   sp = spm.SentencePieceProcessor(model_file='myprefix.model')

   # 分词
   encoded = sp.encode("这是一个测试句子。", out_type=str)
   print(encoded)

   # 还原
   decoded = sp.decode(encoded)
   print(decoded)
   ```

### 应用场景

- **自然语言处理 (NLP)**：广泛应用于各种 NLP 任务，如机器翻译、文本分类、情感分析等。
- **多语言支持**：特别适合处理包含多种语言的文本。
- **低资源语言**：对于那些缺乏丰富词汇资源的语言尤其有用。
- **预训练语言模型**：许多现代预训练语言模型（如 BERT、T5、mBART）都采用了 SentencePiece 作为其分词工具。

### 小结

SentencePiece 是一个强大而灵活的分词框架，适用于广泛的文本处理任务。它的无监督学习特性、语言无关性和高效的子词分词能力使其成为处理复杂和多样化文本数据的理想选择。希望这个简单的介绍能帮助你理解 SentencePiece 的基本概念和应用场景。如果有更多问题或需要进一步的帮助，请随时提问！

## GENE分词器构建

In [ ]:
!pip install sentencepiece

In [ ]:
import sentencepiece as spm

spm.SentencePieceTrainer.train(input='../01-data_env/data/dna_1g.txt,../01-data_env/data/protein_1g.txt',
                               model_prefix='gene_bpe_seg', 
                               vocab_size=60000,
                               model_type='bpe', #默认是unigram
                               num_threads=10,
                              )

In [1]:
from sentencepiece import SentencePieceProcessor
model_path = "gene_bpe_seg.model"
sp_model = SentencePieceProcessor(model_file=model_path)
mm = sp_model.EncodeAsPieces("TCGACGGCACGCGACAGCAGCGAGCCCCGCGCACCCGAGCGCGAKCGFVGPMVHLKVHLEADVASSCRSAVIYLTSEEPFEGVLGLRLKEGIAITGCWPRWPDEMDERSAVWRVEPYTRHFGRVLYSFGV")
print(mm)

['▁TCG', 'ACGGC', 'ACGCG', 'ACAGC', 'AGCG', 'AGCCCC', 'GCGC', 'ACCCG', 'AGCGCG', 'AKCG', 'FVGP', 'MV', 'HLKV', 'HLE', 'ADV', 'ASSC', 'RS', 'AVI', 'YL', 'TS', 'EEP', 'FEG', 'VLGL', 'RLKE', 'GI', 'AI', 'TGC', 'WPR', 'WP', 'DEM', 'DE', 'RS', 'AVW', 'RV', 'EPY', 'TR', 'HFG', 'RVL', 'YS', 'FGV']


## 合并词典到llama

我们以基础版本的llama为例，进行合并，请注意llama的使用限制。

新版本的llama需要自行认证下载。[链接](https://huggingface.co/meta-llama)

```
#建议在终端下执行
pip install -U huggingface_hub
export HF_ENDPOINT=https://hf-mirror.com
huggingface-cli download --resume-download yahma/llama-7b-hf --local-dir llama-7b-hf
```

In [2]:
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"]="python"
from transformers import LlamaTokenizer
from sentencepiece import sentencepiece_model_pb2 as sp_pb2_model
import sentencepiece as spm

In [3]:
llama_tokenizer_dir = "llama-7b-hf" 
dna_sp_model_file = "gene_bpe_seg.model"

# load
llama_tokenizer = LlamaTokenizer.from_pretrained(llama_tokenizer_dir)
dna_sp_model = spm.SentencePieceProcessor()
dna_sp_model.Load(dna_sp_model_file)

llama_spm = sp_pb2_model.ModelProto()
llama_spm.ParseFromString(llama_tokenizer.sp_model.serialized_model_proto())
dna_spm = sp_pb2_model.ModelProto()
dna_spm.ParseFromString(dna_sp_model.serialized_model_proto())

# print number of tokens
print(len(llama_tokenizer),len(dna_sp_model))
print(llama_tokenizer.all_special_tokens)
print(llama_tokenizer.all_special_ids)
print(llama_tokenizer.special_tokens_map)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message


32000 60000
['<s>', '</s>', '<unk>']
[1, 2, 0]
{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}


In [4]:
## Add dna tokens to LLaMA tokenizer
llama_spm_tokens_set=set(p.piece for p in llama_spm.pieces)
print(len(llama_spm_tokens_set))
print(f"Before:{len(llama_spm_tokens_set)}")
for p in dna_spm.pieces:
    piece = p.piece
    score = p.score
    if piece not in llama_spm_tokens_set:
        new_p = sp_pb2_model.ModelProto().SentencePiece()
        new_p.piece = piece
        new_p.score = score # 0?
        llama_spm.pieces.append(new_p)
print(f"New model pieces: {len(llama_spm.pieces)}")

32000
Before:32000
New model pieces: 91643


In [5]:
## Save
output_sp_dir = 'merged_gene_eng_tokenizer_sp'
output_hf_dir = 'merged_gene_eng_tokenizer_hf' # the path to save dna-LLaMA tokenizer
os.makedirs(output_sp_dir,exist_ok=True)
with open(output_sp_dir+'/gene_eng_llama_tokenizer.model', 'wb') as f:
    f.write(llama_spm.SerializeToString())

tokenizer = LlamaTokenizer(vocab_file=output_sp_dir+'/gene_eng_llama_tokenizer.model')
tokenizer.save_pretrained(output_hf_dir)
print(f"gene-LLaMA tokenizer has been saved to {output_hf_dir}")

gene-LLaMA tokenizer has been saved to merged_gene_eng_tokenizer_hf


In [6]:
# Test
llama_tokenizer = LlamaTokenizer.from_pretrained(llama_tokenizer_dir)
dna_llama_tokenizer = LlamaTokenizer.from_pretrained(output_hf_dir)
print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)
print(tokenizer.special_tokens_map)
text='''TCGACGGCACGCGACAGCAGCGAGCCCCGCGCACCCGAGCGCGAKCGFVGPMVHLKVHLEADVASSCRSAVIYLTSEEPFEGVLGLRLKEGIAITGCWPRWPDEMDERSAVWRVEPYTRHFGRVLYSFGV,
The primary use of LLaMA is research on large language models, including'''
print("Test text:\n",text)
print(f"Tokenized by LLaMA tokenizer:{llama_tokenizer.tokenize(text)}")
print(f"Tokenized by GENE-LLaMA tokenizer:{dna_llama_tokenizer.tokenize(text)}")

['<s>', '</s>', '<unk>']
[1, 2, 0]
{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}
Test text:
 TCGACGGCACGCGACAGCAGCGAGCCCCGCGCACCCGAGCGCGAKCGFVGPMVHLKVHLEADVASSCRSAVIYLTSEEPFEGVLGLRLKEGIAITGCWPRWPDEMDERSAVWRVEPYTRHFGRVLYSFGV,
The primary use of LLaMA is research on large language models, including
Tokenized by LLaMA tokenizer:['▁T', 'CG', 'AC', 'G', 'GC', 'AC', 'GC', 'G', 'AC', 'AG', 'CA', 'GC', 'G', 'AG', 'CC', 'CC', 'GC', 'GC', 'AC', 'CC', 'GA', 'GC', 'GC', 'GA', 'K', 'CG', 'F', 'V', 'G', 'PM', 'V', 'HL', 'K', 'V', 'H', 'LE', 'AD', 'VA', 'SS', 'CR', 'S', 'AV', 'I', 'Y', 'LT', 'SEE', 'PF', 'EG', 'V', 'L', 'GL', 'RL', 'KE', 'G', 'IA', 'IT', 'GC', 'W', 'PR', 'WP', 'DE', 'MD', 'ERS', 'AV', 'WR', 'VE', 'PY', 'TR', 'H', 'F', 'GR', 'V', 'LY', 'SF', 'GV', ',', '<0x0A>', 'The', '▁primary', '▁use', '▁of', '▁L', 'La', 'MA', '▁is', '▁research', '▁on', '▁large', '▁language', '▁models', ',', '▁including']
Tokenized by GENE-LLaMA tokenizer:['▁TCG', 'ACGGC', 'ACGCG', 'ACAG', 'CA', '